# Time Series Analysis and Rolling Statistics

This notebook demonstrates time series analysis and rolling window calculations using Polars.

In [ ]:
import os
import polars as pl
import matplotlib.pyplot as plt


## 1. Load Benchmark Data


In [ ]:
datasets_dir = os.path.abspath(os.path.join('..', 'datasets'))
benchmarks = pl.read_csv(f'{datasets_dir}/benchmarks.csv')
benchmarks = benchmarks.with_columns([pl.col('date').str.strptime(pl.Datetime, format='%Y-%m-%dT%H:%M:%S%.f').alias('date')])
benchmarks = benchmarks.sort('date')
benchmarks.head()


## 2. Rolling Statistics
Calculate rolling mean and standard deviation.

In [ ]:
rolling_stats = benchmarks.with_columns([
    pl.col('benchmark_return').rolling_mean(window_size=7).alias('rolling_mean_7d'),
    pl.col('benchmark_return').rolling_std(window_size=30).alias('rolling_std_30d')
])
print(rolling_stats.tail())


## 3. Exponentially Weighted Moving Average (EWMA)


In [ ]:
rolling_stats = rolling_stats.with_columns([pl.col('benchmark_return').ewm_mean(alpha=0.2).alias('ewma_alpha_0.2')])
rolling_stats.tail()


## 4. Save Results


In [ ]:
rolling_stats.write_parquet(f'{datasets_dir}/rolling_benchmarks.parquet')
print('Saved rolling statistics to parquet.')
